In [ ]:
import os
import json
import pandas as pd

from pydub import AudioSegment

In [ ]:
dfpath = '/Users/f004p74/Dartmouth College Dropbox/Lindsey Tepfer/ctom/data/ratings/v1/'
files = [x for x in os.listdir(dfpath) if '.csv' in x]
files.sort()
print(len(files)

First, I calculate how much time has passed at each state in the experiment, creating a new column for each rating dataframe.

In [ ]:
for i in files:
    try:
        df = pd.read_csv(dfpath+i)
        values = []
        counter = 0

        for ix in range(df.index.max()+1):
            counter += df.loc[ix, "RT"]
            values.append(counter)

        df["TIME"] = values

        df.to_csv(dfpath+i)
    except:
        print(i, "no RT")

Then, I find the timepoint in which each pair will (approximately) start their conversation trial by retreiving the 14th state (here, it's the 15th row, simply because it's shifted by 1) which is about where each pair should be ready to talk about their target. 

Importantly, this will not be exact -- most of the audio recordings were started before they began the experiment. This timepoint is a safe estimate to a) remove audio that is unrelated to the experiment (e.g., researcher instructions) and b) not accidentally trim out any of the conversation. However, this timepoint will still facilitate data processing by removing noise and reducing the size of each file.  

In [ ]:
audio_data = '/Volumes/LT/phd/ctom/raw_audio'
pair_list = [x for x in os.listdir(audio_data) if 'pair' in x]
pair_list.sort()
print(len(pair_list))

In [ ]:
df = pd.DataFrame(columns=[["pair", 'trim']])

for pairid in pair_list:

    try:

        trim_data = []

        subjects = [x for x in os.listdir(audio_data+os.sep+pairid+os.sep) if '.WAV' in x]
        subid = subjects[0].split('.')[0].lower()

        datafiles = [x for x in os.listdir(dfpath) if subid+'.csv' in x]

        pair_df = pd.read_csv(dfpath+datafiles[0])

        trim_time = pair_df.loc[15, "TIME"]

        trim_data.extend([pairid, trim_time])

        df.loc[len(df.index)] = trim_data
    
    except:
        print('no TIME column for pair: ', pairid)

Now, I read in the dataframe that has the trim information for each pair, and will use pydub to automatically trim the beginnings of both audio files so that they are temporally aligned. I will later manually trim the ends of the files as well, since they do not need to be the exact same, and will also reduce computational workload on our transcriptions.  

In [ ]:
audio_data = '/Volumes/LT/phd/ctom/raw_audio'
pair_list = [x for x in os.listdir(audio_data) if 'pair' in x]
pair_list.sort()
print(len(pair_list))

df = pd.read_csv('/Users/f004p74/Dartmouth College Dropbox/Lindsey Tepfer/ctom/data/audio/v1_trim_times.csv')

In [ ]:
out_path = '/Volumes/LT/phd/ctom/trimmed_audio/'

for i in range(df.index.max()+1):
    try:
        pair = df.loc[i, "pair"]
        start_time = df.loc[i, "trim"]
        print(pair, start_time)

        pair_files = os.listdir(audio_data+os.sep+pair+os.sep)

        for wav in pair_files:
            fname = wav.split('.')[0]
            audio = AudioSegment.from_file(audio_data+os.sep+pair+os.sep+wav, format="WAV") 
            audio_clipped = audio[start_time:] 
            audio_clipped.export(out_path+pair+os.sep+fname+".WAV", format="WAV") 
    except:
        print("skipping:", pair)

In [ ]:
# need to manually do pair-1 since I didn't log their reaction times for the first experiment!

out_path = '/Volumes/LT/phd/ctom/trimmed_audio/'

pair = 'pair-1'
start_time = 240000
print(pair, start_time)

pair_files = [x for x in os.listdir(audio_data+os.sep+pair+os.sep) if 'SUB' in x]

for wav in pair_files:
    fname = wav.split('.')[0]
    audio = AudioSegment.from_file(audio_data+os.sep+pair+os.sep+wav, format="WAV") 
    audio_clipped = audio[start_time:] 
    audio_clipped.export(out_path+pair+os.sep+fname+".WAV", format="WAV") 